In [1]:
from tut import *
wave, homeserver, mqtt = Initialize("foobot")

publishing entity


Light(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00K\x00\x00\x00x\x08\x06\x00\x00\x0…

Switch(value=False, button_style='success', description='Turn Light On')

Thermostat(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xdc\x00\x00\x04\x04\x08\x02…

publishing entity
publishing entity


In [2]:
# This function is just for the demo so that if you restart the notebook kernel, you
# get the same entity back. Normally you would use:
# entity = wave.CreateEntity(wv.CreateEntityParams())
entity, _ = createOrLoadEntity(wave, "myEntity")
# a perspective is the form of an entity you use in calls to WAVE
perspective = wv.Perspective(entitySecret=wv.EntitySecret(DER=entity.SecretDER))

publishing entity


In [3]:
# test encryption/decryption
msg = "hello world"
uri = "smarthome/light/result"
encryptResult = wave.EncryptMessage(
    wv.EncryptMessageParams(
        namespace=homeserver.namespace(), 
        resource=uri, 
        content=bytes(msg,"utf8")))
print(encryptResult)

# try decrypt it
decryptResult1 = wave.DecryptMessage(
    wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= encryptResult.ciphertext,
        resyncFirst= True))

print (decryptResult1)

# grant permissions from the home server to us
homeserver.grant_permissions_to(entity.hash)

# try decrypt again
decryptResult2 = wave.DecryptMessage(
    wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= encryptResult.ciphertext,
        resyncFirst= True))

print (decryptResult2)

ciphertext: "(\202\004\021\006\n+\006\001\004\001\203\217U\002\005\240\202\004\0010\202\003\375\004\033\274\013B\352&Qhe\252\251p>A?\316B\374H\355`X\305\017\316\216\313,0\202\003\332(\202\003\326\006\n+\006\001\004\001\203\217U\020\002\240\202\003\3060\202\003\302\004\202\002\341\374\325:U\2651\\\024M]\270\236\033-\337*i\035t\226\260\203\302\276\320&\003\272I>\256\027:\257\030x}\323:\362\266\026\345\244mF7(\010\023\267\363a\231DK@GS\351\177\227\037\222\352\030\340\034.qRm\304O\026[:\234\273\302\247\243\006m\203e\304\244sd\315\2345o\n\010\3202m}+\311,.\032\300\r\263I\314\303wUwx3`\344 \006\211\267%[\007\263\036\016\330>`\327e\325\223\034q\017\321\243\016Lm \264q\366\317\323K\246)v\264)\321\307\355|MCQ\334{rmR6\214c\361\347\273\357\347\336o\324\022\374\367\3420w\260\036\n\003\262b@Gx?c\316\204\346\224\2037\315\036\013\326Y\345\324\305#\312\351\201\237-\347\364\356\262\261\261VA\2007\"\t /\220\024D\316\207\327\252qy\312g*~\216S\260\340\261\033!\003\017\372\233 \265\351\2074\256\307\325?;\

In [ ]:

entity = agent.CreateEntity(wv.CreateEntityParams())
if entity.error.code != 0:
    raise Exception(entity.error)
agent.PublishEntity(wv.PublishEntityParams(DER=entity.PublicDER))
perspective=wv.Perspective(
    entitySecret=wv.EntitySecret(DER=entity.SecretDER)
)
proof = agent.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=hs.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["actuate"],
            resource="smarthome/light/control",
        )
    ]
))
# should have no permissions
print ("first attempt without permissions:")
print (proof.error)

hs.grant_permissions_to(entity.hash)
proof2 = agent.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=hs.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        ),
      wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["read"],
            resource="smarthome/light/report",
        )
    ]
))
if proof2.error.code != 0:
    raise Exception(proof2.error)

In [2]:
packed = pack_payload(proof2.proofDER, json.dumps({'state': 'on'}))
client.publish("michael/smarthome/light/control", packed)

In [3]:
packed = pack_payload(proof3.proofDER, json.dumps({'hsp': 68}))
client.publish("michael/smarthome/thermostat/control", packed)

michael/smarthome/light/control
got light command

michael/smarthome/thermostat/control
thermostat command {'hsp': 70, 'csp': 78}
michael/smarthome/light/control
got light command

michael/smarthome/thermostat/control
thermostat command {'hsp': 68}
